In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Databricks works on spark engine. Installing pyspark in your notebook
!pip install pyspark

**Databricks Topics for Friday 7/24**
1. Loading Data from DataFrame to csv file as output.
2. Cleaning files using lambda functions (LTRIM, SUBSTR, INSTR etc.,)
3. Working with JSON files
4. Regular expressions

In [ ]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

In [ ]:
#Pandas reading file
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv
df = pd.read_csv("/kaggle/input/onlineretail/OnlineRetail.csv",sep=',',encoding = 'unicode_escape')

In [ ]:
inv_df = df[df.InvoiceNo =='536365']

In [ ]:
inv_df

In [ ]:
#Pandas to_csv
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html
compression_opts = dict(method='zip',archive_name='clean_out.csv')  
inv_df.to_csv('/kaggle/working/inv_out.zip', index=False,compression=compression_opts)

In [ ]:
#Start spark session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


In [ ]:
#Pyspark read csv file
srk_df = spark.read.csv("/kaggle/input/onlineretail/OnlineRetail.csv",sep=',',header = True)

In [ ]:
srk_df.show()

In [ ]:
sprk_fil_df = srk_df.filter(srk_df.InvoiceNo=='536365')

In [ ]:
sprk_fil_df.show()

In [ ]:
#Spark to Csv file
#sprk_fil_df.write.csv('/kaggle/working/spark_write.csv',sep = ',',header=True)
sprk_fil_df.write.mode('overwrite').option('header', 'true').csv('/kaggle/working/spark_write_2.csv')

In [ ]:
#spark read file
sprk_7_df = spark.read.csv("../input/demo-spark/part-00000-0e499c3f-9c1b-4b9b-bbe5-7d063617a0e4-c000.csv",sep=',',header = True)

In [ ]:
sprk_7_df.show()

In [ ]:
#Cleaning file 
#https://spark.apache.org/docs/2.2.0/api/R/regexp_replace.html
clean_df = sprk_7_df.withColumn('StockCode', regexp_replace('StockCode', "'", '')) \
                 .withColumn('Description', regexp_replace('Description', "@", ''))\
                 .withColumn('Country', regexp_replace('Country', ' ', ''))

In [ ]:
clean_df.show()

In [ ]:
clean_df = clean_df.withColumn(col, func.ltrim(func.rtrim(clean_df[col])))

In [ ]:
clean_df.show()

In [ ]:
subtr_df= clean_df.withColumn('InvoiceDate', substring('InvoiceDate', 1, 9))

In [ ]:
subtr_df.show()

In [ ]:
new_df = subtr_df.withColumn('StockCode', regexp_replace('StockCode', "\D", ''))

In [ ]:
new_df.show()